In [1]:
import nltk
import pandas as pd
from nltk.corpus import stopwords
from textblob import Word
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import wordcloud
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split 
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('/content/training.csv')
test = pd.read_csv('/content/test.csv')
validation = pd.read_csv('/content/validation.csv')

In [3]:
train

,text,label
0,i didnt feel humiliated,0.0
1,i can go from feeling so hopeless to so damned...,0.0
2,im grabbing a minute to post i feel greedy wrong,3.0
3,i am ever feeling nostalgic about the fireplac...,2.0
4,i am feeling grouchy,3.0
...,...,...
10346,i guess ive heard enough over the two months b...,3.0
10347,im sure they feel the more caring loving peopl...,2.0
10348,i was feeling melty and miserable enough mysel...,0.0
10349,i am energetically pursuing my goals or i feel...,3.0


In [4]:
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(train['text'].values)
X = tokenizer.texts_to_sequences(train['text'].values)
vocab_size = len(tokenizer.word_index) + 1
X = pad_sequences(X,maxlen=300)

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, 32, input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(176, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(6,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 32)           390176    
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 32)          0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 176)               147136    
                                                                 
 dense (Dense)               (None, 6)                 1062      
                                                                 
Total params: 538,374
Trainable params: 538,374
Non-trainable params: 0
_________________________________________________________________
None


In [6]:
y=pd.get_dummies(train['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)

In [7]:
batch_size=32
model.fit(X_train, y_train, epochs = 5, batch_size=batch_size, verbose = 'auto')

Epoch 1/5
227/227 [==============================] - 230s 997ms/step - loss: 1.5867 - accuracy: 0.3386
Epoch 2/5
227/227 [==============================] - 226s 995ms/step - loss: 1.2341 - accuracy: 0.5416
Epoch 3/5
227/227 [==============================] - 225s 992ms/step - loss: 0.6837 - accuracy: 0.7718
Epoch 4/5
227/227 [==============================] - 226s 995ms/step - loss: 0.4163 - accuracy: 0.8632
Epoch 5/5
227/227 [==============================] - 224s 987ms/step - loss: 0.2597 - accuracy: 0.9158


In [8]:
model.evaluate(X_test,y_test)

98/98 [==============================] - 12s 120ms/step - loss: 0.3614 - accuracy: 0.8764


[0.3614497482776642, 0.8763683438301086]

In [26]:
test_tokenizer = Tokenizer()
test_tokenizer.fit_on_texts(validation["text"])
test_encoded_docs = test_tokenizer.texts_to_sequences(validation["text"])
test_padded_sequence = pad_sequences(test_encoded_docs, maxlen=300)

test_sentiment_label = validation["label"].factorize()
y_t=pd.get_dummies(test['label'])
test_sentiment_label[1]


Int64Index([0, 2, 3, 1, 4, 5], dtype='int64')

In [28]:
arr=["sadness","joy","love","anger","fear","suprised"]
def predict_sentiment(text):
  tw = tokenizer.texts_to_sequences([text])
  tw = pad_sequences(tw,maxlen=300)
  prediction=model.predict(tw)
  #prediction = int(model.predict(tw).round().item())
  print("Predicted label: ", arr[prediction.argmax()])
test_sentence1 = "I enjoyed my journey on this flight."
predict_sentiment(test_sentence1)
test_sentence2 = "This is the worst flight experience of my life!"
predict_sentiment(test_sentence2)
test_sentence3 = "I can't believe they did that"
predict_sentiment(test_sentence3)
test_sentence4 = "Deadline is approaching."
predict_sentiment(test_sentence4)

Predicted label:  joy
Predicted label:  anger
Predicted label:  love
Predicted label:  fear


In [33]:
test_sentence5 = "I am sorry for her loss."
predict_sentiment(test_sentence5)

Predicted label:  sadness


In [45]:
test_sentence6 = "I could not believe my eyes"
predict_sentiment(test_sentence6)

Predicted label:  fear


In [48]:
test_sentence7 = "She is looking pretty today."
predict_sentiment(test_sentence7)

Predicted label:  joy
